In [ ]:
#default_exp constants.modification

In [ ]:
# hide
__file__ = '../../alphabase/constants/modification.py'

In [ ]:
#export


In [ ]:
#export
import os
import numba
import numpy as np
import pandas as pd
from typing import Union
from copy import deepcopy

from alphabase.yaml_utils import load_yaml
from alphabase.constants.element import calc_formula_mass

_base_dir = os.path.dirname(__file__)

def load_mod_yaml(yaml_file):
    global MOD_INFO_DICT
    global MOD_CHEM
    global MOD_MASS
    global MOD_LOSS_MASS
    global MOD_DF
    MOD_INFO_DICT = load_yaml(yaml_file)
    
    # Add lower-case modifications for future usages
    for key, modinfo in list(MOD_INFO_DICT.items()):
       modname, site = key.split('@')
       if len(site) == 1:
           MOD_INFO_DICT[modname+'@'+site.lower()] = deepcopy(modinfo)
           MOD_INFO_DICT[modname+'@'+site.lower()]['classification'] = 'Lower-case'

    MOD_CHEM = {}
    for key, val in MOD_INFO_DICT.items():
        MOD_CHEM[key] = val['composition']
        MOD_INFO_DICT[key]['unimod_mass'] = MOD_INFO_DICT[key]['mono_mass']
        MOD_INFO_DICT[key]['unimod_modloss'] = MOD_INFO_DICT[key]['modloss']
    

    MOD_MASS = dict(
        [
            (mod, calc_formula_mass(chem))
            for mod, chem in MOD_CHEM.items()
        ]
    )

    MOD_LOSS_MASS = dict(
        [
        (mod, calc_formula_mass(val['modloss_composition']))
        for mod, val in MOD_INFO_DICT.items()
        ]
    )

    MOD_DF = pd.DataFrame().from_dict(MOD_INFO_DICT, orient='index')
    MOD_DF['mass'] = pd.DataFrame().from_dict(MOD_MASS, orient='index')
    MOD_DF['modloss'] = pd.DataFrame().from_dict(MOD_LOSS_MASS, orient='index')

load_mod_yaml(
    os.path.join(_base_dir,
    'used_mod.yaml')
)

def load_modloss_importance(yaml_file):
    global MOD_LOSS_IMPORTANCE
    MOD_LOSS_IMPORTANCE = load_yaml(yaml_file)
    MOD_DF['modloss_importance'] = pd.DataFrame().from_dict(MOD_LOSS_IMPORTANCE, orient='index')
    MOD_DF.loc[pd.isna(MOD_DF['modloss_importance']), 'modloss_importance'] = 0


load_modloss_importance(
    os.path.join(_base_dir,
    'modloss_importance.yaml')
)

In [ ]:
MOD_DF

,avge_mass,classification,composition,modloss,modloss_composition,mono_mass,unimod_id,unimod_mass,unimod_modloss,mass,modloss_importance
GlyGly@K,114.042927,Post-translational,H(6)C(4)N(2)O(2),114.042922,H(6)C(4)N(2)O(2),114.042927,-1,114.042927,114.042927,114.042922,1e6
15N-oxobutanoic@C^Any N-term,-18.023900,Artefact,H(-3)15N(-1),0.000000,,-18.023584,1419,-18.023584,0.000000,-18.023583,0
15N-oxobutanoic@S^Protein N-term,-18.023900,Post-translational,H(-3)15N(-1),0.000000,,-18.023584,1419,-18.023584,0.000000,-18.023583,0
15N-oxobutanoic@T^Protein N-term,-18.023900,Post-translational,H(-3)15N(-1),0.000000,,-18.023584,1419,-18.023584,0.000000,-18.023583,0
2-dimethylsuccinyl@C,144.125300,Chemical derivative,H(8)C(6)O(4),0.000000,,144.042259,1262,144.042259,0.000000,144.042253,0
...,...,...,...,...,...,...,...,...,...,...,...
spermidine@q,128.215300,Lower-case,H(16)C(7)N(2),0.000000,,128.131349,1421,128.131349,0.000000,128.131340,0
spermine@q,185.309700,Lower-case,H(23)C(10)N(3),0.000000,,185.189198,1420,185.189198,0.000000,185.189185,0
sulfo+amino@y,95.077800,Lower-case,H(1)N(1)O(3)S(1),0.000000,,94.967714,997,94.967714,0.000000,94.967710,0
thioacylPA@k,159.206200,Lower-case,H(9)C(6)N(1)O(2)S(1),0.000000,,159.035399,967,159.035399,0.000000,159.035393,0


In [ ]:
for mod, val in MOD_INFO_DICT.items():
    if abs(val['unimod_mass']-MOD_MASS[mod]) > 1e-5:
        print(f"{mod}: unimod mod={val['unimod_mass']}, formula mass={MOD_MASS[mod]}")

4AcAllylGal@C: unimod mod=372.142033, formula mass=372.1420173
ADP-Ribosyl@C: unimod mod=541.06111, formula mass=541.0610899000001
ADP-Ribosyl@D: unimod mod=541.06111, formula mass=541.0610899000001
ADP-Ribosyl@E: unimod mod=541.06111, formula mass=541.0610899000001
ADP-Ribosyl@K: unimod mod=541.06111, formula mass=541.0610899000001
ADP-Ribosyl@N: unimod mod=541.06111, formula mass=541.0610899000001
ADP-Ribosyl@R: unimod mod=541.06111, formula mass=541.0610899000001
ADP-Ribosyl@S: unimod mod=541.06111, formula mass=541.0610899000001
ADP-Ribosyl@T: unimod mod=541.06111, formula mass=541.0610899000001
AHA-Alkyne-KDDDD@M: unimod mod=695.280074, formula mass=695.2800427999999
AHA-SS_CAM@M: unimod mod=252.097088, formula mass=252.09707669999997
AMTzHexNAc2@N: unimod mod=502.202341, formula mass=502.2023182
AMTzHexNAc2@S: unimod mod=502.202341, formula mass=502.2023182
AMTzHexNAc2@T: unimod mod=502.202341, formula mass=502.2023182
AROD@C: unimod mod=820.336015, formula mass=820.3359780999999

In [ ]:
#export

def get_modification_mass(
    peplen:int, 
    mod_names:list, 
    mod_sites:Union[list, np.array]
):
    masses = np.zeros(peplen)
    for site, mod in zip(mod_sites, mod_names):
        if site == 0:
            masses[site] += MOD_MASS[mod]
        elif site == -1:
            masses[site] += MOD_MASS[mod]
        else:
            masses[site-1] += MOD_MASS[mod]
    return masses


In [ ]:
seq = 'AGHCEWQMK'
mod_names = ['Acetyl@Protein N-term', 'Carbamidomethyl@C', 'Oxidation@M']
mod_sites = [0, 4, 8]

get_modification_mass(len(seq), mod_names, mod_sites)

array([42.0105633,  0.       ,  0.       , 57.0214611,  0.       ,
        0.       ,  0.       , 15.9949141,  0.       ])

In [ ]:
%timeit get_modification_mass(len(seq), mod_names, mod_sites)

2.06 µs ± 93.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
#export
@numba.jit(nopython=True, nogil=True)
def _get_modloss(mod_losses, _loss_importance):
    prev_importance = _loss_importance[0]
    prev_most = 0
    for i, _curr_imp in enumerate(_loss_importance[1:]):
        if _curr_imp > prev_importance:
            prev_most = i+1
            prev_importance = _curr_imp
        else:
            mod_losses[i+1] = mod_losses[prev_most]
    return mod_losses

def get_modloss_mass(
    peplen: int, 
    mod_names: list, 
    mod_sites: Union[list, np.array],
    for_nterm_frag: bool,
):
    if not mod_names: return np.zeros(peplen-1)
    mod_losses = np.zeros(peplen+2)
    mod_losses[mod_sites] = [MOD_LOSS_MASS[mod] for mod in mod_names]
    _loss_importance = np.zeros(peplen+2)
    _loss_importance[mod_sites] = [
        MOD_LOSS_IMPORTANCE[mod] if mod in MOD_LOSS_IMPORTANCE else 0 
        for mod in mod_names
    ]
    
    # Will not consider the modloss if the corresponding modloss_importance is 0
    mod_losses[_loss_importance==0] = 0

    if for_nterm_frag:
        return _get_modloss(mod_losses, _loss_importance)[1:-2]
    else:
        return _get_modloss(mod_losses[::-1], _loss_importance[::-1])[-3:0:-1]


In [ ]:
mod_names = ['Oxidation@M', 'Phospho@S', 'Carbamidomethyl@C']
mod_sites = [0, 4, 8]
MOD_LOSS_IMPORTANCE['Oxidation@M'] = 10
get_modloss_mass(10, mod_names, mod_sites, True)

array([63.9982825, 63.9982825, 63.9982825, 97.9768922, 97.9768922,
       97.9768922, 97.9768922, 97.9768922, 97.9768922])

In [ ]:
MOD_LOSS_IMPORTANCE['Oxidation@M'] = 0
get_modloss_mass(10, mod_names, mod_sites, True)

array([ 0.       ,  0.       ,  0.       , 97.9768922, 97.9768922,
       97.9768922, 97.9768922, 97.9768922, 97.9768922])

In [ ]:
MOD_LOSS_IMPORTANCE['Oxidation@M'] = 10
get_modloss_mass(10, mod_names, mod_sites, False)

array([97.9768922, 97.9768922, 97.9768922,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ])

### Note that get_modloss_mass is a little bit time comsuming
`%timeit get_modloss_mass(10, mod_names, mod_sites, False)`

`Results (12 seconds in total): 12.6 µs ± 96.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)`